# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started. Once you have completed the code you can download the notebook for making a submission.

In [2]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [5]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [22]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())

In [23]:
# no of images to be considered per video
x = 30

# Dimensions of image
y=120
z=120

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [24]:
def generator(source_path, folder_list, batch_size):
 
    print('Source path = ', source_path, '; batch size =', batch_size)

    # Sampling x evenly spaced frames from each video
    img_idx = np.linspace(0, 29, x).astype(int)  

    def process_batch(start_idx, end_idx):
        current_batch_size = end_idx - start_idx
        batch_data = np.zeros((current_batch_size, x, y, z, 3))  # Preallocate for RGB images
        batch_labels = np.zeros((current_batch_size, 5))  # Assuming 5 classes for one-hot encoding

        for folder_idx in range(current_batch_size):
            folder_path = source_path + '/' + folder_list[start_idx + folder_idx].split(';')[0]
            label = int(folder_list[start_idx + folder_idx].strip().split(';')[2])

            imgs = sorted(os.listdir(folder_path))  # Sort to ensure frame order consistency

            for idx, item in enumerate(img_idx):
                image_path = folder_path + '/' + imgs[item]
                image = imread(image_path).astype(np.float32)
                
                # Cropping (example: crop the center region)
                crop_size = min(image.shape[:2])  # Get the smaller dimension (assuming the crop is square)
                center = (image.shape[0] // 2, image.shape[1] // 2)  # Find the center of the image
                crop_image = image[
                    center[0] - crop_size // 2 : center[0] + crop_size // 2, 
                    center[1] - crop_size // 2 : center[1] + crop_size // 2
                ]

                    
                # Resize the image to (y, z)
                image_resized = resize(crop_image, (y, z))

                # Normalizing and Assigning each channel explicitly
                batch_data[folder_idx, idx, :, :, 0] = image_resized[:, :, 0] / 255.0  # Normalize R channel
                batch_data[folder_idx, idx, :, :, 1] = image_resized[:, :, 1] / 255.0  # Normalize G channel
                batch_data[folder_idx, idx, :, :, 2] = image_resized[:, :, 2] / 255.0  # Normalize B channel

            batch_labels[folder_idx, label] = 1  # One-hot encode the label

        return batch_data, batch_labels

    while True:
        t = np.random.permutation(folder_list)  # Shuffle the folder list
        num_batches = len(folder_list) // batch_size  # Calculate the number of full batches

        for batch in range(num_batches):
            start_idx = batch * batch_size
            end_idx = start_idx + batch_size
            yield process_batch(start_idx, end_idx)

        # Handle the remaining data points after full batches
        remaining_folders = len(folder_list) % batch_size
        if remaining_folders > 0:
            start_idx = num_batches * batch_size
            end_idx = len(folder_list)
            yield process_batch(start_idx, end_idx)


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [25]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

# training sequences = 663
# validation sequences = 100


In [26]:
batch_size = 30
num_epochs = 5
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [27]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [28]:
# clearing the cache
import gc
tf.keras.backend.clear_session()
gc.collect()

77

In [33]:
# All imports
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, LSTM,GRU,TimeDistributed, Dense,Dropout,Input, BatchNormalization, Activation, GlobalAveragePooling3D,GlobalAveragePooling2D,Bidirectional
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,EarlyStopping

from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from keras import optimizers

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [34]:
def create_conv3d_model(input_shape, num_classes):

    model = Sequential()

    # Conv3D Block 1
    model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01),input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Dropout(0.5))

    # Conv3D Block 2
    model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu',kernel_regularizer=l2(0.01), padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Dropout(0.5))

    # Conv3D Block 3
    model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu',kernel_regularizer=l2(0.01), padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Dropout(0.5))

    # Global Average Pooling (instead of Flatten)
    model.add(GlobalAveragePooling3D())
    
    # Fully Connected Layer
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.7))

    # Output Layer
    model.add(Dense(num_classes, activation='softmax'))

    return model

input_shape = (x,y,z, 3)  # (frames, height, width, channels)
num_classes = 5

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [35]:
model = create_conv3d_model(input_shape, num_classes)

optimizer = Adam(learning_rate=0.001)  # You can adjust the learning rate as needed
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_3 (Conv3D)           (None, 30, 120, 120, 64)  5248      
                                                                 
 batch_normalization_3 (Batc  (None, 30, 120, 120, 64)  256      
 hNormalization)                                                 
                                                                 
 max_pooling3d_3 (MaxPooling  (None, 15, 60, 60, 64)   0         
 3D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 15, 60, 60, 64)    0         
                                                                 
 conv3d_4 (Conv3D)           (None, 15, 60, 60, 128)   221312    
                                                                 
 batch_normalization_4 (Batc  (None, 15, 60, 60, 128) 

In [36]:
# callbacks
model_name = 'Conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto',save_freq = 'epoch')

LR = ReduceLROnPlateau(
    monitor='val_loss',       # Metric to monitor (e.g., validation loss)
    factor=0.5,               # Factor by which the learning rate will be reduced
    patience=3,               # Number of epochs with no improvement before reducing the learning rate
    min_lr=1e-6,              # Lower bound on the learning rate
    verbose=1                 # Print updates when learning rate changes
)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

callbacks_list = [early_stopping,checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [24]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 10
Epoch 1/5
67/67 [==============================] - ETA: 0s - loss: 3.2749 - categorical_accuracy: 0.3891Source path =  /home/datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to Conv3D_2024-12-0921_30_00.698437/model-00001-3.27491-0.38914-3.44845-0.21000.h5
67/67 [==============================] - 175s 3s/step - loss: 3.2749 - categorical_accuracy: 0.3891 - val_loss: 3.4485 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 2/5
67/67 [==============================] - ETA: 0s - loss: 2.0318 - categorical_accuracy: 0.4329
Epoch 00002: saving model to Conv3D_2024-12-0921_30_00.698437/model-00002-2.03182-0.43288-3.11014-0.22000.h5
67/67 [==============================] - 159s 2s/step - loss: 2.0318 - categorical_accuracy: 0.4329 - val_loss: 3.1101 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 3/5
67/67 [==============================] - ETA: 0s - loss: 1.6520 - categorical_accuracy: 0.4917
E

In [50]:
# creating 2 seperate CNN and RNN layers with transfer learning from Resnet50
# Freezing Resnet and using simple RNN

def create_transfer_cnn_lstm_model(input_shape, num_classes):
    # Input shape is (number_of_frames, height, width, channels)
    inputs = Input(shape=input_shape)

    # Pre-trained ResNet50 model
    base_model = ResNet50(
        weights="imagenet", include_top=False, input_shape=(input_shape[1], input_shape[2], input_shape[3])
    )
    base_model.trainable = False  # Freeze the pre-trained weights

    # CNN Module using TimeDistributed
    cnn_model = Sequential(name="CNN_Module")
    cnn_model.add(base_model)
    cnn_model.add(GlobalAveragePooling2D())

    time_distributed_cnn = TimeDistributed(cnn_model, name="TimeDistributed_CNN")(inputs)

    # RNN Module
    lstm_layer = LSTM(128, return_sequences=False, dropout=0.5, recurrent_dropout=0.3, name="LSTM_Layer")(
        time_distributed_cnn
    )
    outputs = Dense(num_classes, activation="softmax", name="Output_Layer")(lstm_layer)

    # Final Model
    model = Model(inputs=inputs, outputs=outputs, name="ResNet50_CNN_RNN_Model")

    return model

In [51]:
model2 = create_transfer_cnn_lstm_model(input_shape, num_classes)
optimizer = Adam(learning_rate=0.001)  # You can adjust the learning rate as needed
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

Model: "ResNet50_CNN_RNN_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 120, 120, 3)  0         
                             ]                                   
                                                                 
 TimeDistributed_CNN (TimeDi  (None, 30, 2048)         23587712  
 stributed)                                                      
                                                                 
 LSTM_Layer (LSTM)           (None, 128)               1114624   
                                                                 
 Output_Layer (Dense)        (None, 5)                 645       
                                                                 
Total params: 24,702,981
Trainable params: 1,115,269
Non-trainable params: 23,587,712
_________________________________________________________________
None


In [ ]:
model2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 30
Epoch 1/5
23/23 [==============================] - ETA: 0s - loss: 1.6186 - categorical_accuracy: 0.2443Source path =  /home/datasets/Project_data/val ; batch size = 30

Epoch 00001: val_loss improved from inf to 1.56808, saving model to Conv3D_2024-12-1001_09_04.144053/model-00001-1.61855-0.24434-1.56808-0.24000.h5
23/23 [==============================] - 155s 7s/step - loss: 1.6186 - categorical_accuracy: 0.2443 - val_loss: 1.5681 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 2/5
23/23 [==============================] - ETA: 0s - loss: 1.5947 - categorical_accuracy: 0.2579
Epoch 00002: val_loss improved from 1.56808 to 1.50348, saving model to Conv3D_2024-12-1001_09_04.144053/model-00002-1.59475-0.25792-1.50348-0.24000.h5
23/23 [==============================] - 149s 7s/step - loss: 1.5947 - categorical_accuracy: 0.2579 - val_loss: 1.5035 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 3/5
20/23 [=======

In [52]:
# changes to above model
# allowing last 20 layers of Resnet to learn
# Using BiDirenctional LSTM insted of simple LSTM

def create_transfer_cnn_bi_lstm_model(input_shape, num_classes):
    # Input shape is (number_of_frames, height, width, channels)
    inputs = Input(shape=input_shape)

    # Pre-trained ResNet50 model
    base_model = ResNet50(
        weights="imagenet", include_top=False, input_shape=(input_shape[1], input_shape[2], input_shape[3])
    )
    base_model.trainable = False  # Freeze the pre-trained weights

    # Unfreeze the last few layers for fine-tuning
    for layer in base_model.layers[-20:]:
        layer.trainable = True

    # CNN Module with L2 regularization
    cnn_model = Sequential(name="CNN_Module")
    cnn_model.add(base_model)
    cnn_model.add(GlobalAveragePooling2D())
    cnn_model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
    cnn_model.add(Dropout(0.5))

    # TimeDistributed CNN Layer
    time_distributed_cnn = TimeDistributed(cnn_model, name="TimeDistributed_CNN")(inputs)

    # RNN Module with Bidirectional LSTM and L2 regularization
    lstm_layer = Bidirectional(LSTM(256, return_sequences=False, dropout=0.5, recurrent_dropout=0.3, kernel_regularizer=l2(0.01)), name="Bidirectional_LSTM_Layer")(time_distributed_cnn)
    outputs = Dense(num_classes, activation="softmax", name="Output_Layer")(lstm_layer)

    # Final Model
    model = Model(inputs=inputs, outputs=outputs, name="ResNet50_CNN_RNN_Model")

    return model

In [53]:
model3 = create_transfer_cnn_bi_lstm_model(input_shape, num_classes)
optimizer = Adam(learning_rate=0.001)  # You can adjust the learning rate as needed
model3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model3.summary())

Model: "ResNet50_CNN_RNN_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 30, 120, 120, 3)  0         
                             ]                                   
                                                                 
 TimeDistributed_CNN (TimeDi  (None, 30, 256)          24112256  
 stributed)                                                      
                                                                 
 Bidirectional_LSTM_Layer (B  (None, 512)              1050624   
 idirectional)                                                   
                                                                 
 Output_Layer (Dense)        (None, 5)                 2565      
                                                                 
Total params: 25,165,445
Trainable params: 1,577,733
Non-trainable params: 23,587,712
________________________

In [102]:
model3.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 30
Epoch 1/5
23/23 [==============================] - 159s 7s/step - loss: 10.5335 - categorical_accuracy: 0.3439 - val_loss: 7.5909 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 2/5
23/23 [==============================] - 149s 7s/step - loss: 5.3837 - categorical_accuracy: 0.4766 - val_loss: 4.6400 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 3/5
23/23 [==============================] - 148s 7s/step - loss: 3.2338 - categorical_accuracy: 0.6139 - val_loss: 3.5458 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 4/5
23/23 [==============================] - 143s 6s/step - loss: 2.3732 - categorical_accuracy: 0.6335 - val_loss: 3.2592 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 5/5
23/23 [==============================] - 149s 7s/step - loss: 1.9887 - categorical_accuracy: 0.6591 - val_loss: 5.0402 - val_categorical_accuracy: 0.2300 - lr: 0.0010


In [54]:
# changes done
# allowing 40 layers of resnet to learn
# increased params for Dense layer
def create_transfer_cnn_bi_lstm_model2(input_shape, num_classes):
    # Input shape is (number_of_frames, height, width, channels)
    inputs = Input(shape=input_shape)

    # Pre-trained ResNet50 model
    base_model = ResNet50(
        weights="imagenet", include_top=False, input_shape=(input_shape[1], input_shape[2], input_shape[3])
    )
    base_model.trainable = False  # Freeze the pre-trained weights

    # Unfreeze the last few layers for fine-tuning
    for layer in base_model.layers[-40:]:
        layer.trainable = True

    # CNN Module with L2 regularization
    cnn_model = Sequential(name="CNN_Module")
    cnn_model.add(base_model)
    cnn_model.add(GlobalAveragePooling2D())
    cnn_model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
    cnn_model.add(Dropout(0.4))

    # TimeDistributed CNN Layer
    time_distributed_cnn = TimeDistributed(cnn_model, name="TimeDistributed_CNN")(inputs)

    # RNN Module with Bidirectional LSTM and L2 regularization
    lstm_layer = Bidirectional(LSTM(256, return_sequences=False, dropout=0.4, recurrent_dropout=0.4, kernel_regularizer=l2(0.01)), name="Bidirectional_LSTM_Layer")(time_distributed_cnn)
    outputs = Dense(num_classes, activation="softmax", name="Output_Layer")(lstm_layer)

    # Final Model
    model = Model(inputs=inputs, outputs=outputs, name="ResNet50_CNN_RNN_Model")

    return model

In [55]:
model4 = create_transfer_cnn_bi_lstm_model2(input_shape, num_classes)
optimizer = Adam(learning_rate=0.001)  # You can adjust the learning rate as needed
model4.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model4.summary())

Model: "ResNet50_CNN_RNN_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 30, 120, 120, 3)  0         
                             ]                                   
                                                                 
 TimeDistributed_CNN (TimeDi  (None, 30, 512)          24636800  
 stributed)                                                      
                                                                 
 Bidirectional_LSTM_Layer (B  (None, 512)              1574912   
 idirectional)                                                   
                                                                 
 Output_Layer (Dense)        (None, 5)                 2565      
                                                                 
Total params: 26,214,277
Trainable params: 2,626,565
Non-trainable params: 23,587,712
________________________

In [117]:
model4.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 30
Epoch 1/5
23/23 [==============================] - 159s 7s/step - loss: 17.1959 - categorical_accuracy: 0.3092 - val_loss: 11.7062 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 2/5
23/23 [==============================] - 150s 7s/step - loss: 8.2201 - categorical_accuracy: 0.4751 - val_loss: 6.3495 - val_categorical_accuracy: 0.1900 - lr: 0.0010
Epoch 3/5
23/23 [==============================] - 150s 7s/step - loss: 4.6493 - categorical_accuracy: 0.5445 - val_loss: 4.4078 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 4/5
23/23 [==============================] - 146s 7s/step - loss: 3.2007 - categorical_accuracy: 0.6018 - val_loss: 3.4146 - val_categorical_accuracy: 0.2700 - lr: 0.0010
Epoch 5/5
23/23 [==============================] - 154s 7s/step - loss: 2.4814 - categorical_accuracy: 0.6501 - val_loss: 2.7880 - val_categorical_accuracy: 0.3300 - lr: 0.0010


In [56]:
def create_transfer_cnn_bi_gru_model(input_shape, num_classes):
    # Input layer
    inputs = Input(shape=input_shape)

    # Pre-trained ResNet50 model (transfer learning)
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(input_shape[1], input_shape[2], input_shape[3]))
    base_model.trainable = False
    
    # Unfreeze the last few layers for fine-tuning
    for layer in base_model.layers[-20:]:
        layer.trainable = True

    # CNN Module with L2 regularization
    cnn_model = Sequential(name="CNN_Module")
    cnn_model.add(base_model)
    cnn_model.add(GlobalAveragePooling2D())
    cnn_model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
    cnn_model.add(Dropout(0.4))

    # TimeDistributed CNN Layer
    time_distributed_cnn = TimeDistributed(cnn_model, name="TimeDistributed_CNN")(inputs)

    # First Bidirectional GRU Layer
    gru_layer_1 = Bidirectional(GRU(256, return_sequences=True, dropout=0.4, recurrent_dropout=0.4, kernel_regularizer=l2(0.01)), name="Bidirectional_GRU_Layer_1")(time_distributed_cnn)

    # Second Bidirectional GRU Layer
    gru_layer_2 = Bidirectional(GRU(128, return_sequences=False, dropout=0.4, recurrent_dropout=0.4, kernel_regularizer=l2(0.01)), name="Bidirectional_GRU_Layer_2")(gru_layer_1)

    # Output layer
    outputs = Dense(num_classes, activation="softmax", name="Output_Layer")(gru_layer_2)

    # Final Model
    model = Model(inputs=inputs, outputs=outputs, name="ResNet50_CNN_BiGRU_Model")

    return model

In [57]:
model5 = create_transfer_cnn_bi_gru_model(input_shape, num_classes)
optimizer = Adam(learning_rate=0.001)  # You can adjust the learning rate as needed
model5.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model5.summary())

Model: "ResNet50_CNN_BiGRU_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 30, 120, 120, 3)  0         
                             ]                                   
                                                                 
 TimeDistributed_CNN (TimeDi  (None, 30, 512)          24636800  
 stributed)                                                      
                                                                 
 Bidirectional_GRU_Layer_1 (  (None, 30, 512)          1182720   
 Bidirectional)                                                  
                                                                 
 Bidirectional_GRU_Layer_2 (  (None, 256)              493056    
 Bidirectional)                                                  
                                                                 
 Output_Layer (Dense)        (None, 5)    

In [15]:
model5.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 30
Epoch 1/5


2024-12-10 00:16:56.425665: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


23/23 [==============================] - ETA: 0s - loss: 22.9844 - categorical_accuracy: 0.3107Source path =  /home/datasets/Project_data/val ; batch size = 30

Epoch 00001: val_loss improved from inf to 15.65347, saving model to Conv3D_2024-12-1000_15_50.721424/model-00001-22.98441-0.31071-15.65347-0.34000.h5


/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


23/23 [==============================] - 165s 7s/step - loss: 22.9844 - categorical_accuracy: 0.3107 - val_loss: 15.6535 - val_categorical_accuracy: 0.3400 - lr: 0.0010
Epoch 2/5
23/23 [==============================] - ETA: 0s - loss: 11.5936 - categorical_accuracy: 0.4751
Epoch 00002: val_loss improved from 15.65347 to 8.91108, saving model to Conv3D_2024-12-1000_15_50.721424/model-00002-11.59361-0.47511-8.91108-0.18000.h5
23/23 [==============================] - 153s 7s/step - loss: 11.5936 - categorical_accuracy: 0.4751 - val_loss: 8.9111 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 3/5
23/23 [==============================] - ETA: 0s - loss: 6.6962 - categorical_accuracy: 0.6063
Epoch 00003: val_loss improved from 8.91108 to 5.94155, saving model to Conv3D_2024-12-1000_15_50.721424/model-00003-6.69616-0.60633-5.94155-0.26000.h5
23/23 [==============================] - 151s 7s/step - loss: 6.6962 - categorical_accuracy: 0.6063 - val_loss: 5.9416 - val_categorical_accuracy:

In [37]:
def create_transfer_cnn_bi_gru_model_final(input_shape, num_classes):
    # Input layer
    inputs = Input(shape=input_shape)

    # Pre-trained ResNet50 model (transfer learning)
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(input_shape[1], input_shape[2], input_shape[3]))
    base_model.trainable = False
    
    # Unfreeze the last few layers for fine-tuning
    for layer in base_model.layers[-30:]:
        layer.trainable = True

    # CNN Module with L2 regularization
    cnn_model = Sequential(name="CNN_Module")
    cnn_model.add(base_model)
    cnn_model.add(GlobalAveragePooling2D())
    cnn_model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.05)))
    cnn_model.add(Dropout(0.5))

    # TimeDistributed CNN Layer
    time_distributed_cnn = TimeDistributed(cnn_model, name="TimeDistributed_CNN")(inputs)

    # First Bidirectional GRU Layer
    gru_layer_1 = Bidirectional(GRU(512, return_sequences=True, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=l2(0.05)), name="Bidirectional_GRU_Layer_1")(time_distributed_cnn)

    # Second Bidirectional GRU Layer
    gru_layer_2 = Bidirectional(GRU(216, return_sequences=False, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=l2(0.05)), name="Bidirectional_GRU_Layer_2")(gru_layer_1)

    # Output layer
    outputs = Dense(num_classes, activation="softmax", name="Output_Layer")(gru_layer_2)

    # Final Model
    model = Model(inputs=inputs, outputs=outputs, name="ResNet50_CNN_BiGRU_Model")

    return model

In [38]:
model6 = create_transfer_cnn_bi_gru_model_final(input_shape, num_classes)
optimizer = Adam(learning_rate=1e-4)
model6.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model6.summary())

Model: "ResNet50_CNN_BiGRU_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 120, 120, 3)  0         
                             ]                                   
                                                                 
 TimeDistributed_CNN (TimeDi  (None, 30, 512)          24636800  
 stributed)                                                      
                                                                 
 Bidirectional_GRU_Layer_1 (  (None, 30, 1024)         3151872   
 Bidirectional)                                                  
                                                                 
 Bidirectional_GRU_Layer_2 (  (None, 432)              1609632   
 Bidirectional)                                                  
                                                                 
 Output_Layer (Dense)        (None, 5)    

In [39]:
model6.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 30
Epoch 1/5
23/23 [==============================] - ETA: 0s - loss: 188.8110 - categorical_accuracy: 0.2881Source path =  /home/datasets/Project_data/val ; batch size = 30

Epoch 00001: val_loss improved from inf to 177.43866, saving model to Conv3D_2024-12-1002_48_49.316964/model-00001-188.81100-0.28808-177.43866-0.17000.h5
23/23 [==============================] - 167s 7s/step - loss: 188.8110 - categorical_accuracy: 0.2881 - val_loss: 177.4387 - val_categorical_accuracy: 0.1700 - lr: 1.0000e-04
Epoch 2/5
23/23 [==============================] - ETA: 0s - loss: 168.2739 - categorical_accuracy: 0.3288
Epoch 00002: val_loss improved from 177.43866 to 158.11235, saving model to Conv3D_2024-12-1002_48_49.316964/model-00002-168.27394-0.32881-158.11235-0.23000.h5
23/23 [==============================] - 151s 7s/step - loss: 168.2739 - categorical_accuracy: 0.3288 - val_loss: 158.1124 - val_categorical_accuracy: 0.2300 - lr: 1